In [1]:
import pandas as pd
products = pd.read_csv('data_skinrec_bismillah.csv')
products.head()

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src,sensitive,...,refreshing,hydrating,balancing,pore_care,no_whitecast,oil_control,black_spot,brightening,soothing,anti_aging
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,209000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,0,...,0,0,0,0,0,0,0,1,0,0
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",181800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,0,...,0,0,1,0,0,0,0,0,1,0
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",149000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",1,...,0,0,1,0,0,0,0,0,1,0
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,290000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,0,...,0,0,0,0,0,0,0,1,0,0
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",194650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",0,...,0,0,0,0,0,0,0,1,1,0


In [3]:
print(products.columns)

Index(['product_href', 'product_name', 'product_type', 'brand',
       'notable_effects', 'skintype', 'price', 'description', 'picture_src',
       'sensitive', 'combination', 'oily', 'dry', 'normal', 'rating',
       'keluhan', 'kulit_kusam', 'jerawat', 'bekas_jerawat', 'pori_pori_besar',
       'flek_hitam', 'garis_halus_dan_kerutan', 'komedo',
       'warna_kulit_tidak_merata', 'kemerahan', 'kulit_kendur', 'moisturizing',
       'skin_barrier', 'uv_protection', 'acne_free', 'refreshing', 'hydrating',
       'balancing', 'pore_care', 'no_whitecast', 'oil_control', 'black_spot',
       'brightening', 'soothing', 'anti_aging'],
      dtype='object')


In [43]:
def recommend_skincare(product_type, skin_types, skin_problems, notable_effects):
    # Filter produk berdasarkan product_type
    filtered_df = products[products['product_type'] == product_type]
    
    # Filter produk berdasarkan skin types yang sesuai (nilai 1 hanya untuk skin_types yang dipilih)
    all_skin_types = ['sensitive', 'combination', 'dry', 'oily', 'normal']
    other_skin_types = [stype for stype in all_skin_types if stype not in skin_types]
    
    # Produk harus memiliki nilai 1 untuk semua skin_types yang dipilih dan 0 untuk lainnya
    skin_type_mask = (filtered_df[skin_types].sum(axis=1) == len(skin_types)) & (filtered_df[other_skin_types].sum(axis=1) == 0)
    filtered_df = filtered_df[skin_type_mask]
    
    # Hitung jumlah 1 di kolom skin problems yang sesuai dengan pilihan user
    total_skin_problems = filtered_df[['kulit_kusam', 'jerawat', 'bekas_jerawat', 'pori_pori_besar', 'flek_hitam',
                                       'garis_halus_dan_kerutan', 'komedo', 'warna_kulit_tidak_merata', 'kemerahan',
                                       'kulit_kendur']].sum(axis=1)
    skin_problems_match = filtered_df[skin_problems].sum(axis=1) / total_skin_problems
    
    # Hitung jumlah 1 di kolom notable effects yang sesuai dengan pilihan user
    total_notable_effects = filtered_df[['acne_free', 'soothing', 'brightening', 'moisturizing', 'hydrating',
                                         'pore_care', 'anti_aging', 'balancing', 'uv_protection', 'skin_barrier',
                                         'refreshing', 'oil_control', 'no_whitecast', 'black_spot']].sum(axis=1)
    notable_effects_match = filtered_df[notable_effects].sum(axis=1) / total_notable_effects
    
    # Gabungkan semua skor kesesuaian
    total_score = (skin_problems_match + notable_effects_match)/2
    
    # Urutkan produk berdasarkan total_score secara descending
    recommended_products = filtered_df.assign(total_score=total_score).sort_values(by='total_score', ascending=False)
    recommended_products = recommended_products[recommended_products['total_score'] > 0]
    
    return recommended_products[['product_name', 'brand', 'price', 'description', 'picture_src', 'total_score', 'rating', 'product_type']]

# Contoh penggunaan fungsi recommend_skincare dengan multiple input
skin_types = ['oily']
skin_problems = ['bekas_jerawat', 'jerawat', 'flek_hitam']
notable_effects = ['skin_barrier', 'oil_control', 'pore_care', 'brightening']

recommended_products = recommend_skincare('Face Wash', skin_types, skin_problems, notable_effects)
recommended_products

,product_name,brand,price,description,picture_src,total_score,rating,product_type
0,ACWELL Bubble Free PH Balancing Cleanser,ACWELL,209000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,1.00,5,Face Wash
722,NIVEA Sparkling White Whitening Facial foam,NIVEA,23625,NIVEA Sparkling WHITE FOAM 100ML Sehari-hari t...,https://www.beautyhaul.com/assets/uploads/prod...,0.75,5,Face Wash
361,EVERSHINE Moringa Facial Wash,EVERSHINE,129000,"Evershine Facial Wash dengan sakura extract ,C...",https://www.beautyhaul.com/assets/uploads/prod...,0.50,5,Face Wash
370,First Lab Probiotic Deep Cleansing Foam 140ml,FIRST LAB,270600,All FIRST LAB's Skincare are using a Patented ...,https://www.beautyhaul.com/assets/uploads/prod...,0.50,5,Face Wash
469,HUMPHREY Charcoal Natural Facial Wash,HUMPHREY,44900,Pembersih wajah dan make up yang alami terbuat...,https://www.beautyhaul.com/assets/uploads/prod...,0.50,5,Face Wash
713,NAUFA Face Soap 50 gr,NAUFA,20900,"Alcohol Free, based on customer's review, this...",https://www.beautyhaul.com/assets/uploads/prod...,0.50,5,Face Wash
737,Npure Marigold Deep Cleansiing Foaming,N'PURE,129000,Npure Marigold Deep Cleansing Foaming Face Was...,https://www.beautyhaul.com/assets/uploads/prod...,0.50,5,Face Wash
738,Npure Cica Series Face Wash,N'PURE,90000,1st local brand with real cica leaves We proud...,https://www.beautyhaul.com/assets/uploads/prod...,0.50,5,Face Wash
